<a href="https://colab.research.google.com/github/Sakitha/Unstractured_PDF_DATA/blob/master/pharser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
print("Version: ", tf.__version__)
import tensorflow_addons as tf_ad
import numpy as np

Version:  2.2.0


In [ ]:
class FFD(tf.keras.Model):

  def __init__(self):
    
    super(FFD, self).__init__()

    self.dense1 = tf.keras.layers.Dense(4, activation=tf.nn.relu)
    self.dense2 = tf.keras.layers.Dense(3, activation=tf.nn.softmax)

  def call(self, inputs, dropout_vec):
    
    inputs = tf.where(tf.math.is_nan(inputs), tf.zeros_like(inputs), inputs).numpy()# replace nan with zeros
    dropout_vec = tf.convert_to_tensor(dropout_vec, dtype= inputs.dtype)

    i = tf.math.multiply(inputs, dropout_vec)
    x = self.dense1(i)
    
    return self.dense2(x)


class NerModel(tf.keras.Model):
  def __init__(self, hidden_num, label_size):
    super(NerModel, self).__init__()

    self.ffd = FFD()

    self.num_hidden = hidden_num
    self.label_size = label_size

    self.biLSTM = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(hidden_num, input_shape=(4,), return_sequences=True), )
      
    self.dense = tf.keras.layers.Dense(label_size,activation='softmax')

    self.transition_params = tf.Variable(tf.random.uniform(shape=(label_size, label_size)))


  def call(self,inputs,labels=None,training=None):
    
    outputs = []


    for time_step in range(inputs.shape[2]-1):
  
      input = inputs[:, time_step, :]
      dropout_vec = 1 - np.isnan(input).astype(np.int)

      ffd_output = self.ffd(input, dropout_vec)
      
      outputs.append(ffd_output)

    logits = tf.stack(tf.unstack(tf.stack(outputs,axis = 0),axis=1),axis=0)

      
    #text_lens = tf.math.reduce_sum(tf.cast(tf.math.not_equal(inputs, 0), dtype=tf.int32), axis=-1)

    if labels is not None:
      
      label_sequences = tf.convert_to_tensor(labels, dtype=tf.int32)
      log_likelihood, self.transition_params = tf_ad.text.crf_log_likelihood(logits,label_sequences,[3,3],transition_params=self.transition_params))
      
      return logits, text_lens, log_likelihood
    else:

      return logits, text_lens

  

In [ ]:
#model = NerModel(hidden_num = args.hidden_num, vocab_size = len(vocab2id), label_size= len(tag2id), embedding_size = args.embedding_size)
model = NerModel(hidden_num = 1, label_size= 3)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

In [ ]:
def train_one_step(inputs, targets):
  with tf.GradientTape() as tape:
      logits, text_lens= model(inputs, targets, training=True)
      loss = - tf.reduce_mean(logits)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  return loss,logits, text_lens

In [ ]:
## Note: Rerunning this cell uses the same model variables
inputs = np.asarray([[[.1,.2,.3,.4],[.1,.2,.3,.4],[.1,.2,.3,.4]],[[.1,.2,.3,.4],[.1,.2,.3,.4],[.1,.2,.3,.4]]], dtype=np.float32)

targets   = np.asarray([[1,2,2],[1,6,2]])

# Keep results for plotting
train_loss_results = []
train_accuracy_results = []

num_epochs = 5

for epoch in range(num_epochs):
  epoch_loss_avg = tf.keras.metrics.Mean()
  epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

  # Training loop - using batches of 32
  
    # Optimize the model
  loss, logits, text_lens = train_one_step(inputs, targets)
  #logits, text_lens = train_one_step(inputs, targets)

    # Track progress
  epoch_loss_avg.update_state(loss)  # Add current batch loss
    # Compare predicted label to actual label
    # training=True is needed only if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
  #epoch_accuracy.update_state(targets, model(inputs, training=True))

  # End epoch
  train_loss_results.append(epoch_loss_avg.result())s=self.transition_params
  #train_accuracy_results.append(epoch_accuracy.result())

  
  print("Epoch {:03d}: Loss: {:.3f}".format(epoch,epoch_loss_avg.result()))

InvalidArgumentError: ignored

In [ ]:
inputs = np.asarray([[[.1,.2,.3,.4],[.1,.2,.3,.4],[.1,.2,.3,.4]]], dtype=np.float32)

biLSTM = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(2, return_sequences=True), )
dense = tf.keras.layers.Dense(3,activation='softmax')

logits = dense(biLSTM(inputs))
print(logits)

tf.Tensor(
[[[0.33443305 0.3122755  0.35329145]
  [0.33616656 0.31302205 0.35081142]
  [0.33652422 0.31749234 0.3459835 ]]], shape=(1, 3, 3), dtype=float32)


In [ ]:
inputs = tf.constant([[[.1,.2,.3,.4],[.1,.2,.3,.4],[.1,.2,.3,.4]]], dtype=np.float32)
targets   = np.asarray([[1,2,0]])
 
log_likelihood, transition_params = tf_ad.text.crf_log_likelihood(logits, targets,[3]) #[batch_size, max_seq_len]

In [ ]:
inputs = tf.constant([[[ 0.1, 0.2, 0.3, 0.4],[ 0.1, 0.2, 0.3, 0.4],[ 0.1, 0.2, 0.3, 0.4]],[[ 0.1, 0.2, 0.3, 0.4],[ 0.1, 0.2, 0.3, 0.4],[ 0.1, 0.2, 0.3, 0.4]]])
print(tf.shape(inputs))
targets   = tf.constant([[1,2,0],[1,1,0]])
print(tf.shape(targets))
text_lens = [2,2]

tf.Tensor([2 3 4], shape=(3,), dtype=int32)
tf.Tensor([2 3], shape=(2,), dtype=int32)


In [ ]:
biLSTM = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(3, return_sequences=True), )
dense = tf.keras.layers.Dense(3,activation='softmax')

logits = dense(biLSTM(inputs))
print(logits)

tf.Tensor(
[[[0.37104094 0.31334856 0.31561047]
  [0.35531765 0.32940105 0.3152813 ]
  [0.33627197 0.34606826 0.31765977]]

 [[0.37104094 0.31334856 0.31561047]
  [0.35531765 0.32940105 0.3152813 ]
  [0.33627197 0.34606826 0.31765977]]], shape=(2, 3, 3), dtype=float32)


In [ ]:
log_likelihood, transition_params = tf_ad.text.crf_log_likelihood(logits, targets ,text_lens)

In [ ]:
inputs = tf.constant([[[ 0.1, 0.2, 0.3, 0.4],[ 0.1, 0.2, 0.3, 0.4],[ 0.1, 0.2, 0.3, 0.4]],[[ 0.1, 0.2, 0.3, 0.4],[ 0.1, 0.2, 0.3, 0.4],[ 0.1, 0.2, 0.3, 0.4]]])
text_lens = tf.math.reduce_sum(tf.cast(tf.math.not_equal(inputs, 0), dtype=tf.int32), axis=-1)
print(text_lens)

tf.Tensor(
[[4 4 4]
 [4 4 4]], shape=(2, 3), dtype=int32)


In [ ]:
#biLSTM = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(hidden_num, input_shape=(4,), return_sequences=True), )
#logits = dense(biLSTM(inputs))

inputs = tf.constant([[[ 0.1, np.nan, 0.3, 0.4],[ 0.2, 0.3, 0.4, 0.4],[ 0.1, 0.4, 0.4, 0.4]],[[ 0.4, 0.2, 0.3, 0.4],[ 0.1, 0.4, 0.3, 0.4],[ 0.1, 0.2, 0.4, 0.4]]])

class FFD(tf.keras.Model):

  def __init__(self):
    
    super(FFD, self).__init__()

    self.dense1 = tf.keras.layers.Dense(4, activation=tf.nn.relu)
    self.dense2 = tf.keras.layers.Dense(3, activation=tf.nn.softmax)

  def call(self, inputs, dropout_vec):

    inputs = tf.where(tf.math.is_nan(inputs), tf.zeros_like(inputs), inputs).numpy()# replace nan with zeros
    dropout_vec = tf.convert_to_tensor(dropout_vec, dtype= inputs.dtype)

    i = tf.math.multiply(inputs, dropout_vec)
    x = self.dense1(i)
    
    return self.dense2(x)


ffd = FFD()

outputs = []


for time_step in range(inputs.shape[2]-1):
  
  input = inputs[:, time_step, :]
  dropout_vec = 1 - np.isnan(input).astype(np.int)
  #inputs[np.isnan(inputs)] = 0 #this should handle in the model.

  ffd_output = ffd(input, dropout_vec)
  outputs.append(ffd_output) #since batch sice is 1

In [ ]:
outputs

[<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
 array([[       nan,        nan,        nan],
        [0.3439042 , 0.3394569 , 0.31663895]], dtype=float32)>,
 <tf.Tensor: shape=(2, 3), dtype=float32, numpy=
 array([[0.35776103, 0.34707984, 0.2951592 ],
        [0.3508219 , 0.34332076, 0.30585733]], dtype=float32)>,
 <tf.Tensor: shape=(2, 3), dtype=float32, numpy=
 array([[0.36040863, 0.34848273, 0.2911086 ],
        [0.35815635, 0.3472904 , 0.2945532 ]], dtype=float32)>]

In [ ]:
logits = tf.stack(tf.unstack(tf.stack(outputs,axis = 0),axis=1),axis=0)

In [ ]:
targets   = tf.constant([[1,2,0]])
text_lens = [3]
log_likelihood, transition_params = tf_ad.text.crf_log_likelihood(logits, targets ,text_lens)

In [ ]:
x = np.asarray([[np.nan, 1, 3]])
#x = x[np.logical_not(np.isnan(x))]
x = 1 - np.isnan(x).astype(np.int)

x[np.isnan(x)] = 0
#B.astype(np.int)
x

array([[0, 1, 1]])

In [ ]:
inputs = tf.constant([[[ 0.1, np.nan, 0.0, 0.4],[ 0.2, 0.3, 0.4, 0.4],[ 0.1, 0.4, 0.4, 0.4]],[[ 0.4, 0.2, 0.3, 0.4],[ 0.1, 0.4, 0.3, 0.4],[ 0.1, 0.2, 0.4, 0.4]]])

input = inputs[:, 0, :]
dropout_vec = 1 - np.isnan(input).astype(np.int)

dense1 = tf.keras.layers.Dense(4, activation=tf.nn.relu)
dense2 = tf.keras.layers.Dense(3, activation=tf.nn.softmax)

input = tf.where(tf.math.is_nan(input), tf.zeros_like(input), input).numpy()

i = tf.math.multiply(input, tf.convert_to_tensor(dropout_vec, dtype= input.dtype))
j = dense1(i)
k = tf.math.multiply(j, tf.convert_to_tensor(dropout_vec, dtype=tf.float32))


#x = j(i)

#dense2(x)

In [ ]:
x k 
  v
l

x,v,j,l

model 1 = just a FFN, model 2 = lstm cell
[[x_x, x_x, f_x, x_v, x_v, f_v, 0, 0, 0 ,x_l, x_l, f_l ],
[x_x, x_x, f_x, x_v, x_v, f_v, 0, 0, 0 ,x_l, x_l, f_l ],
[x_x, x_x, f_x, x_v, x_v, f_v, 0, 0, 0 ,x_l, x_l, f_l ],
[x_x, x_x, f_x, x_v, x_v, f_v, 0, 0, 0 ,x_l, x_l, f_l ]]

model 2 = lstm loop
[[x_x, x_x, f_x, x_v, x_v, f_v, [0, 0, 0] ,x_l, x_l, f_l ],
[x_x, x_x, f_x, x_v, x_v, f_v, [0, 0, 0],x_l, x_l, f_l ],
[x_x, x_x, f_x, x_v, x_v, f_v, [0, 0, 0] ,x_l, x_l, f_l ],
[x_x, x_x, f_x, x_v, x_v, f_v, [0, 0, 0] ,x_l, x_l, f_l ]]



<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[0.1, 0. , 0. , 0.4],
       [0.4, 0.2, 0.3, 0.4]], dtype=float32)>

In [ ]:
j

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.07846069, 0.        ]], dtype=float32)>

In [ ]:
class FFD(tf.keras.Model):

  def __init__(self):
    
    super(FFD, self).__init__()

    self.dense1 = tf.keras.layers.Dense(4, activation=tf.nn.relu)
    self.dense2 = tf.keras.layers.Dense(3, activation=tf.nn.softmax)

  def call(self, inputs, dropout_vec):
    
    inputs = tf.where(tf.math.is_nan(inputs), tf.zeros_like(inputs), inputs).numpy()# replace nan with zeros
    dropout_vec = tf.convert_to_tensor(dropout_vec, dtype= inputs.dtype)

    i = tf.math.multiply(inputs, dropout_vec)
    x = self.dense1(i)
    
    return self.dense2(x)


class NerModel(tf.keras.Model):
  def __init__(self, hidden_num, label_size):
    super(NerModel, self).__init__()

    self.ffd = FFD()

    self.num_hidden = hidden_num
    self.label_size = label_size

    self.biLSTM = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(hidden_num, input_shape=(4,), return_sequences=True), )
      
    self.dense = tf.keras.layers.Dense(label_size,activation='softmax')

    self.transition_params = tf.Variable(tf.random.uniform(shape=(label_size, label_size)))


  def call(self,inputs,labels=None,training=None):
    
    outputs = []


    for time_step in range(inputs.shape[2]-1):
  
      input = inputs[:, time_step, :]
      dropout_vec = 1 - np.isnan(input).astype(np.int)

      ffd_output = self.ffd(input, dropout_vec)
      
      outputs.append(ffd_output)

    logits = tf.stack(tf.unstack(tf.stack(outputs,axis = 0),axis=1),axis=0)

      
    #text_lens = tf.math.reduce_sum(tf.cast(tf.math.not_equal(inputs, 0), dtype=tf.int32), axis=-1)

    if labels is not None:
      
      label_sequences = tf.convert_to_tensor(labels, dtype=tf.int32)
      log_likelihood, self.transition_params = tf_ad.text.crf_log_likelihood(logits,label_sequences,[3,3],transition_params=self.transition_params))
      
      return logits, text_lens, log_likelihood
    else:

      return logits, text_lens

  

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.07846069, 0.        ]], dtype=float32)>

In [ ]:
samples, timesteps, features = 2, 10, 8
inputs = np.random.random([samples, timesteps, features]).astype(np.float32)
inputs[:, 3, :] = 0.
inputs[:, 5, :] = 0.
inputs

array([[[0.30563   , 0.8578603 , 0.09359249, 0.45120567, 0.80962515,
         0.80292475, 0.718621  , 0.72389615],
        [0.76597154, 0.51600266, 0.6955795 , 0.15295061, 0.17176181,
         0.39968657, 0.77693236, 0.4316625 ],
        [0.8949149 , 0.4862184 , 0.69731045, 0.89588296, 0.44410047,
         0.7779092 , 0.3695223 , 0.70633113],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        ],
        [0.08722161, 0.45591286, 0.76407033, 0.6093148 , 0.35731044,
         0.98193914, 0.0062343 , 0.30492923],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        ],
        [0.85145736, 0.56486636, 0.20516314, 0.5360692 , 0.856764  ,
         0.28055114, 0.23640926, 0.1716804 ],
        [0.937895  , 0.07445275, 0.58404136, 0.08941504, 0.2051817 ,
         0.7395205 , 0.09835254, 0.29336497],
        [0.10854036, 0.62597597, 0.8276086 , 0.00956349, 0.4365472 ,
         0.